In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")

In [3]:
stats

,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,0,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,...,0.0,0.0,Phoenix Suns,41,41,0.500,23.0,104.3,104.0,0.28
1,1,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,...,0.0,0.0,Miami Heat,50,32,0.610,6.0,88.9,86.6,1.73
2,2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,...,0.0,0.0,Dallas Mavericks,24,58,0.293,40.0,90.6,97.0,-6.47
3,3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,...,0.0,0.0,Dallas Mavericks,20,62,0.244,42.0,91.4,97.5,-6.33
4,4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,...,0.0,0.0,Dallas Mavericks,19,31,0.380,18.0,91.6,94.0,-2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16046,16046,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,...,0.0,0.0,Detroit Pistons,50,32,0.610,0.0,91.4,87.7,2.97
16047,16047,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,...,0.0,0.0,Detroit Pistons,50,32,0.610,0.0,94.3,92.2,1.69
16048,16048,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,...,0.0,0.0,Los Angeles Clippers,37,45,0.451,25.0,95.7,96.5,-0.47
16049,16049,Željko Rebrača,C,33,LAC,29,2,14.2,1.8,3.3,...,0.0,0.0,Los Angeles Clippers,47,35,0.573,7.0,97.2,95.6,1.75


In [4]:
del stats["Unnamed: 0"]

In [6]:
pd.isnull(stats).sum()

Player        0
Pos           0
Age           0
Tm            0
G             0
GS            0
MP            0
FG            0
FGA           0
FG%          68
3P            0
3PA           0
3P%        2146
2P            0
2PA           0
2P%         118
eFG%         68
FT            0
FTA           0
FT%         590
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PTS           0
Year          0
Pts Won       0
Pts Max       0
Share         0
Team          0
W             0
L             0
W/L%          0
GB            0
PS/G          0
PA/G          0
SRS           0
dtype: int64

In [ ]:
stats = stats.fillna(0)

In [11]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS'],
      dtype='object')

In [ ]:
predictors = [ 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 
       'W/L%', 'GB', 'PS/G','PA/G', 'SRS']

In [ ]:
train = stats[stats["Year"] < 2024]

In [14]:
test = stats[stats["Year"] == 2025]